## BigQuery Spark Connector

Python does not currently support jar files loaded in an Environment. We load the required library into the session.

<mark>PLEASE VERIFY THE LAKEHOUSE (ABFSS) PATH TO THE JAR IS CORRECT FOR YOUR WORKSPACE</mark>

In [ ]:
%%configure -f: 
{
    "conf": {
        "spark.jars": "abfss://0fb2d96e-53e7-4c49-a594-beb0891ac121@msit-onelake.dfs.fabric.microsoft.com/9c7403b4-606d-412c-9224-64ccdbff9cc3/Files/dependencies/spark-3.5-bigquery-0.37.0.jar"
    }
}  

In [ ]:
config_json_path = "/env/config/fabric-poc-bq-config.json"

In [ ]:
from FabricSync.BQ.Loader import *
from FabricSync.DeltaTableUtility import *

# Metadata Sync

In [ ]:
config_metadata = ConfigMetadataLoader(spark, mssparkutils, config_json_path)
config_metadata.sync_bq_metadata()
config_metadata.auto_detect_table_profiles()

# Scheduler

In [ ]:
scheduler = Scheduler(spark, mssparkutils, config_json_path)
scheduler.run()

# Loader

In [ ]:
schedule_loader = BQScheduleLoader(spark, mssparkutils, config_json_path)
schedule_loader.run_async_schedule()

# Metadata Table Maintenance

In [ ]:
tbls = ["bq_sync_configuration", "bq_sync_schedule", "bq_sync_schedule_telemetry"]

for tbl in tbls:
    table_maint = DeltaTableMaintenance(spark, mssparkutils, tbl)
    table_maint.optimize_and_vacuum()